In [1]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import tensorflow as tf

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# Tensorflow and Deep Learning

In this lab assignment, first you will learn how to build and train a neural network that recognises handwritten digits, and then you will build LeNet-5 CNN architecture, which is widely used for handwritten digit recognition. At the end of this lab assignment, you will make AlexNet CNN architecture, which won the 2012 ImageNet ILSVRC challenge.

---
# 1. Dataset
In the first part of the assignment, we use the MNIST dataset, which is a set of 70,000 small images of digits handwritten by high school students and employees of the US Census Bureau. Each image is labeled with the digit it represents. There are 70,000 images, and each image has 784 features. This is because each image is 28×28=784 pixels, and each feature simply represents one pixel's intensity, from 0 (white) to 255 (black). The following figure shows a few images from the MNIST dataset to give you a feel for the complexity of the classification task.

<img src="figs/1-mnist.png" style="width: 300px;"/>

To begin the assignment, first, use `mnist_data.read_data_sets` and download images and labels. It return two lists, called `mnist.test` with 10K images+labels, and `mnist.train` with 60K images+labels.

In [2]:
# TODO: Replace <FILL IN> with appropriate code

from tensorflow.examples.tutorials.mnist import input_data as mnist_data

mnist = mnist_data.read_data_sets("data/")

X_train = mnist.train.images
Y_train = mnist.train.labels.astype("int")
X_test = mnist.test.images
Y_test = mnist.test.labels.astype("int")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


---
# 2. A One-Layer Neural Network
<img src="figs/2-comic1.png" style="width: 500px;"/>

Let's start by building a one-layer neural network. Handwritten digits in the MNIST dataset are 28x28 pixel greyscale images. The simplest approach for classifying them is to use the 28x28=784 pixels as inputs for a **one-layer neural network**. Each neuron in the network does a weighted sum of all of its inputs, adds a bias and then feeds the result through some non-linear activation function. Here we design a one-layer neural network with 10 output neurons since we want to classify digits into 10 classes (0 to 9).
<img src="figs/3-one_layer.png" style="width: 400px;"/>


For a classification problem, an *activation function* that works well is **softmax**. Applying softmax on a vector is done by taking the exponential of each element and then normalising the vector.
<img src="figs/4-softmax.png" style="width: 300px;"/>

We can summarise the behaviour of this single layer of neurons into a simple formula using a *matrix multiply*. If we give input data into the network in *mini-batch* of 100 images, it produces 100 predictions as the output. We define the **weights matrix $W$** with 10 columns, in which each column indicates the weight of a one class (a single digit), from 0 to 9. Using the first column of $W$, we can compute the weighted sum of all the pixels of the first image. This sum corresponds to the first neuron that points to the number 0. Using the second column of $W$, we do the same for the second neuron (number 1) and so on until the 10th neuron. We can then repeat the operation for the remaining 99 images in the mini-batch. If we call $X$ the matrix containing our 100 images (each row corresponds to one digit), all the weighted sums for our 10 neurons, computed on 100 images are simply $X.W$. Each neuron must now add its bias. Since we have 10 neurons, we have 10 bias constants. We finally apply the **softmax activation function** and obtain the formula describing a one-layer neural network, applied to 100 images.
<img src="figs/5-xw.png" style="width: 600px;"/>
<img src="figs/6-softmax2.png" style="width: 500px;"/>

Then, we need to use the **cross-entropy** to measure how good the predictions are, i.e., the distance between what the network tells us and what we know to be the truth. The cross-entropy is a function of weights, biases, pixels of the training image and its known label. If we compute the partial derivatives of the cross-entropy relatively to all the weights and all the biases, we obtain a **gradient**, computed for a given image, label and present value of weights and biases. We can update weights and biases by a fraction of the gradient and do the same thing again using the next batch of training images.
<img src="figs/7-cross_entropy.png" style="width: 600px;"/>

### Define Variables and Placeholders
First we define TensorFlow **variables** and **placeholders**. *Variables* are all the parameters that you want the training algorithm to determine for you (e.g., weights and biases). *Placeholders* are parameters that will be filled with actual data during training (e.g., training images). The shape of the tensor holding the training images is [None, 28, 28, 1] which stands for:
  - 28, 28, 1: our images are 28x28 (784) pixels x 1 value per pixel (grayscale). The last number would be 3 for color images and is not really necessary here.
  - None: this dimension will be the number of images in the mini-batch. It will be known at training time.

We also need an additional placeholder for the training labels that will be provided alongside training images.

In [3]:
# TODO: Replace <FILL IN> with appropriate code

# neural network with 1 layer of 10 softmax neurons
#
# · · · · · · · · · ·       (input data, flattened pixels)       X [batch, 784] 
# \x/x\x/x\x/x\x/x\x/    -- fully connected layer (softmax)      W [784, 10]     b[10]
#   · · · · · · · ·                                              Y_hat [batch, 10]

# input X: 28x28 grayscale images, the first dimension (None) will index the images in the mini-batch
X = tf.placeholder(shape=[None, 28*28], dtype=tf.float32, name="input")

# correct answers will go here
Y = tf.placeholder(shape=[None], dtype=tf.int32, name="labels")

# weights W[784, 10], 784 = 28 * 28
W = tf.get_variable(name="W", shape=[784, 10], dtype=tf.float32, initializer=tf.initializers.random_normal(),
                    regularizer=None, trainable=True, collections=None)

# biases b[10]
b = tf.get_variable(name="b", dtype=tf.float32, initializer=tf.zeros(shape=[10]),
                    regularizer=None, trainable=True, collections=None)

### Build The Model
Now, we can make a **model** for a one-layer neural network. The formula is the one we explained before, i.e., $\hat{Y} = softmax(X . W + b)$. You can use the `tf.nn.softmax` and `tf.matmul` to build the model. Here, we need to use the `tf.reshape` to transform our 28x28 images into single vectors of 784 pixels.

In [4]:
# TODO: Replace <FILL IN> with appropriate code

# flatten the images into a single line of pixels
XX = tf.reshape(X, [-1, 28**2])

# The model
Y_hat = tf.nn.softmax(tf.matmul(XX, W) + b)

### Define The Cost Function
Now, we have model predictions $\hat{Y}$ and correct labels $Y$, so for each instance $i$ (image) we can compute the cross-entropy as the **cost function**: $cross\_entropy = -\sum(Y_i * log(\hat{Y}i))$. You can use `reduce_mean` to add all the components in a tensor.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

Y_one_hot = tf.cast(tf.one_hot(Y, 10), tf.float32)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y_one_hot * tf.log(Y_hat), 1))

correct = tf.nn.in_top_k(Y_hat, Y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float64))

### Traine the Model
Now, select the gradient descent optimiser `GradientDescentOptimizer` and ask it to minimise the cross-entropy cost. In this step, TensorFlow computes the partial derivatives of the cost function relatively to all the weights and all the biases (the gradient). The gradient is then used to update the weights and biases. Set the learning rate is $0.005$.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
train_step = optimizer.minimize(cross_entropy)

### Execute the Model
It is time to run the training loop. All the TensorFlow instructions up to this point have been preparing a computation graph in memory but nothing has been computed yet. The computation requires actual data to be fed into the placeholders. This is supplied in the form of a Python dictionary, where the keys are the names of the placeholders. During the trainig print out the cost every 200 steps. Moreove, after training the model, print out the accurray of the model by testing it on the test data.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

# init
init = tf.global_variables_initializer()
n_epochs = 5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        _, t_cost = sess.run([train_step, cross_entropy], feed_dict = {X: X_train, Y: Y_train})
        if (i % 200 == 0):
            print("epoch: %d cost: %f" % (i, t_cost))
    acc = sess.run(accuracy, feed_dict = {X: X_test, Y: Y_test})
    print("Final accuracy: %f" % (acc))
        

epoch: 0 cost: 15.751952
epoch: 200 cost: 12.254673
epoch: 400 cost: 10.234916
epoch: 600 cost: 8.828868
epoch: 800 cost: 7.706673
epoch: 1000 cost: 6.787934
epoch: 1200 cost: 6.032645
epoch: 1400 cost: 5.410252
epoch: 1600 cost: 4.899727
epoch: 1800 cost: 4.482111
epoch: 2000 cost: 4.137480
epoch: 2200 cost: 3.849045
epoch: 2400 cost: 3.604397
epoch: 2600 cost: 3.394345
epoch: 2800 cost: 3.212001
epoch: 3000 cost: 3.052174
epoch: 3200 cost: 2.910883
epoch: 3400 cost: 2.785052
epoch: 3600 cost: 2.672286


---
# 2. Add More Layers

<img src="figs/8-comic2.png" style="width: 500px;"/>

Now, let's improve the recognition accuracy by adding more layers to the neural network. The neurons in the second layer, instead of computing weighted sums of pixels will compute weighted sums of neuron outputs from the previous layer. We keep the softmax function as the activation function on the last layer, but on intermediate layers we will use the the **sigmoid** activation function. So, let's build a five-layer fully connected neural network with the following structure, and train the model with the trainging data and print out its accuracy on the test data.
<img src="figs/9-five_layer.png" style="width: 500px;"/>

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

# neural network with five layers
#
# · · · · · · · · · ·          (input data, flattened pixels)       X [batch, 784]   
# \x/x\x/x\x/x\x/x\x/       -- fully connected layer (sigmoid)      W1 [784, 200]      B1 [200]
#  · · · · · · · · ·                                                Y1_hat [batch, 200]
#   \x/x\x/x\x/x\x/         -- fully connected layer (sigmoid)      W2 [200, 100]      B2 [100]
#    · · · · · · ·                                                  Y2_hat [batch, 100]
#     \x/x\x/x\x/           -- fully connected layer (sigmoid)      W3 [100, 60]       B3 [60]
#      · · · · ·                                                    Y3_hat [batch, 60]
#       \x/x\x/             -- fully connected layer (sigmoid)      W4 [60, 30]        B4 [30]
#        · · ·                                                      Y4_hat [batch, 30]
#         \x/               -- fully connected layer (softmax)      W5 [30, 10]        B5 [10]
#          ·                                                        Y_hat [batch, 10]

# to reset the Tensorflow default graph
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(shape=[None, 28*28], dtype=tf.float32, name="input")
Y = tf.placeholder(shape=[None], dtype=tf.int32, name="labels")

# five layers and their number of neurons, i.e., 200, 100, 60, 30, and 10
W1 = tf.get_variable(name="W1", 
                    shape=[784, 200], 
                    dtype=tf.float32,
                    initializer=tf.initializers.random_normal(), 
                    trainable=True)
B1 = tf.get_variable(name="B1", 
                    dtype=tf.float32, 
                    initializer=tf.zeros(shape=[200]),
                    trainable=True)


W2 = tf.get_variable(name="W2", 
                    shape=[200, 100], 
                    dtype=tf.float32,
                    initializer=tf.initializers.random_normal(), 
                    trainable=True)
B2 = tf.get_variable(name="B2", 
                    dtype=tf.float32, 
                    initializer=tf.zeros(shape=[100]),
                    trainable=True)

W3 = tf.get_variable(name="W3", 
                    shape=[100, 60], 
                    dtype=tf.float32,
                    initializer=tf.initializers.random_normal(), 
                    trainable=True)
B3 = tf.get_variable(name="B3", 
                    dtype=tf.float32, 
                    initializer=tf.zeros(shape=[60]),
                    trainable=True)

W4 = tf.get_variable(name="W4", 
                    shape=[60, 30], 
                    dtype=tf.float32,
                    initializer=tf.initializers.random_normal(), 
                    trainable=True)

B4 = tf.get_variable(name="B4", 
                    dtype=tf.float32, 
                    initializer=tf.zeros(shape=[30]),
                    trainable=True)

W5 = tf.get_variable(name="W5", 
                    shape=[30, 10], 
                    dtype=tf.float32,
                    initializer=tf.initializers.random_normal(), 
                    trainable=True)

B5 = tf.get_variable(name="B5", 
                    dtype=tf.float32, 
                    initializer=tf.zeros(shape=[10]),
                    trainable=True)

########################################
# build the model
########################################
XX = tf.reshape(X, [-1, 28**2])

Y1_hat = tf.nn.sigmoid(tf.matmul(XX, W1) + B1)
Y2_hat = tf.nn.sigmoid(tf.matmul(Y1_hat, W2) + B2)
Y3_hat = tf.nn.sigmoid(tf.matmul(Y2_hat, W3) + B3)
Y4_hat = tf.nn.sigmoid(tf.matmul(Y3_hat, W4) + B4)
Y_hat = tf.nn.softmax(tf.matmul(Y4_hat, W5) + B5)

########################################
# define the cost function
########################################
Y_one_hot = tf.cast(tf.one_hot(Y, 10), tf.float32)
cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y_one_hot * tf.log(Y_hat), 1))

correct = tf.nn.in_top_k(Y_hat, Y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float64))
########################################
# define the optimizer
########################################
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
train_step = optimizer.minimize(cross_entropy)

########################################
# execute the model
########################################
init = tf.global_variables_initializer()

n_epochs = 5000
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        _, cost = sess.run([train_step, cross_entropy], feed_dict = {X: X_train, Y: Y_train})
        if (i % 200 == 0):
            print("epoch: %d cost: %f" % (i, cost))
    acc = sess.run(accuracy, feed_dict = {X: X_test, Y: Y_test})
    print("Final accuracy: %f" % (acc))

---
# 3. Special Care for Deep Networks
As layers were added, neural networks tended to converge with more difficulties. For example, the accuracy could stuck at 0.1. Here, we want to apply some updates to the network we built in the previous part to improve its performance. 

### ReLU Activation Function
<img src="figs/10-comic3.png" style="width: 500px;"/>
The sigmoid activation function is actually quite problematic in deep networks. It squashes all values between 0 and 1 and when you do so repeatedly, neuron outputs and their gradients can vanish entirely. An alternative activation function is **ReLU** that shows better performance compare to sigmoid. It looks like as below:
<img src="figs/11-relu.png" style="width: 300px;"/>

### A Better Optimizer
In very high dimensional spaces like here, **saddle points** are frequent. These are points that are not local minima, but where the gradient is nevertheless zero and the gradient descent optimizer stays stuck there. One possible solution to tackle this probelm is to use better optimizers, such as Adam optimizer `tf.train.AdamOptimizer`.

### Random Initialisations
When working with ReLUs, the best practice is to initialise bias values to small positive values, so that neurons operate in the non-zero range of the ReLU initially.

### Learning Rate
<img src="figs/12-comic4.png" style="width: 500px;"/>
With two, three or four intermediate layers, you can now get close to 98% accuracy, if you push the iterations to 5000 or beyond. But, the results are not very consistent, and the curves jump up and down by a whole percent. A good solution is to start fast and decay the learning rate exponentially from $0.005$ to $0.0001$ for example. In order to pass a different learning rate to the `AdamOptimizer` at each iteration, you will need to define a new placeholder and feed it a new value at each iteration through `feed_dict`. Here is the formula for exponential decay: $learning\_rate = lr\_min + (lr\_max - lr\_min) * e^{\frac{-i}{2000}}$, where $i$ is the iteration number.

### NaN?
In the network you built in the last section, you might see accuracy curve crashes and the console outputs NaN for the cross-entropy. It may happen, because you are attempting to compute a $log(0)$, which is indeed Not A Number (NaN). Remember that the cross-entropy involves a log, computed on the output of the softmax layer. Since softmax is essentially an exponential, which is never zero, we should be fine, but with 32 bit precision floating-point operations, exp(-100) is already a genuine zero. TensorFlow has a handy function that computes the softmax and the cross-entropy in a single step, implemented in a numerically stable way. To use it, you will need to separate the weighted sum plus bias on the last layer, before softmax is applied and then give it with the true values to the function `tf.nn.softmax_cross_entropy_with_logits`.

In the code below, apply the following changes and show their impact on the accuracy of the model on training data, as well as the test data:
* Replace the sigmoid activation function with ReLU
* Use the Adam optimizer
* Initialize weights with small random values between -0.2 and +0.2, and make sure biases are initialised with small positive values, for example 0.1
* Update the learning rate in different iterations. Start fast and decay the learning rate exponentially from $0.005$ to $0.0001$, i.e., 
```
max_learning_rate = 0.005
min_learning_rate = 0.0001
decay_speed = 2000.0
```
* Use `tf.nn.softmax_cross_entropy_with_logits` to prevent getting NaN in output.

In [ ]:
def get_weight(name, shape):
    return tf.get_variable(name=name, 
                    shape=shape, 
                    dtype=tf.float32,
                    initializer=tf.initializers.random_uniform(-0.2, 0.2), 
                    trainable=True)

def get_bias(name, shape, val=0.1):
    return tf.get_variable(name=name, 
                    dtype=tf.float32, 
                    initializer=tf.constant(val, shape=shape),
                    trainable=True)

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

# neural network with 5 layers
#
# · · · · · · · · · ·          (input data, flattened pixels)       X [batch, 784]   
# \x/x\x/x\x/x\x/x\x/       -- fully connected layer (sigmoid)      W1 [784, 200]      B1[200]
#  · · · · · · · · ·                                                Y1_hat [batch, 200]
#   \x/x\x/x\x/x\x/         -- fully connected layer (sigmoid)      W2 [200, 100]      B2[100]
#    · · · · · · ·                                                  Y2_hat [batch, 100]
#     \x/x\x/x\x/           -- fully connected layer (sigmoid)      W3 [100, 60]       B3[60]
#      · · · · ·                                                    Y3_hat [batch, 60]
#       \x/x\x/             -- fully connected layer (sigmoid)      W4 [60, 30]        B4[30]
#        · · ·                                                      Y4_hat [batch, 30]
#         \x/               -- fully connected layer (softmax)      W5 [30, 10]        B5[10]
#          ·                                                        Y_hat [batch, 10]

# to reset the Tensorflow default graph
reset_graph()
########################################
# define variables and placeholders
########################################
X = tf.placeholder(shape=[None, 28*28], dtype=tf.float32, name="input")
Y = tf.placeholder(shape=[None], dtype=tf.int32, name="labels")

# variable learning rate
learning_rate = tf.placeholder(shape=(), dtype=tf.float32, name="learning_rate")

# five layers and their number of neurons, i.e., 200, 100, 60, 30, and 10
# when using RELUs, make sure biases are initialised with small positive values, for example 0.1
W1 = get_weight("W1", [28*28, 200])
B1 = get_bias("B1", [200])

W2 = get_weight("W2", [200, 100])
B2 = get_bias("B2", [100])

W3 = get_weight("W3", [100, 60])
B3 = get_bias("B3", [60])

W4 = get_weight("W4", [60, 30])
B4 = get_bias("B4", [30])

W5 = get_weight("W5", [30, 10])
B5 = get_bias("B5", [10])

########################################
# build the model
########################################
XX = tf.reshape(X, [-1, 28**2])

Y1_hat = tf.nn.relu(tf.matmul(XX, W1) + B1)
Y2_hat = tf.nn.relu(tf.matmul(Y1_hat, W2) + B2)
Y3_hat = tf.nn.relu(tf.matmul(Y2_hat, W3) + B3)
Y4_hat = tf.nn.relu(tf.matmul(Y3_hat, W4) + B4)
Y_hat = tf.matmul(Y4_hat, W5) + B5

########################################
# defining the cost function
########################################
Y_one_hot = tf.cast(tf.one_hot(Y, 10), tf.float32)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y_one_hot, logits=Y_hat)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optimizer
########################################
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(cross_entropy)

correct = tf.nn.in_top_k(Y_hat, Y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float64))
########################################
# execute the model
########################################
init = tf.global_variables_initializer()

n_epochs = 5000
max_learning_rate = 0.005
min_learning_rate = 0.0001
decay_speed = 2000.0

def get_lr(min, max, decay_speed, i):
    return min + (max - min)*np.exp(-i / decay_speed)

with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        lr = get_lr(min_learning_rate, max_learning_rate, decay_speed, i)
        _, tr_cost, tr_acc = sess.run([train_step, cross_entropy, accuracy], 
                           feed_dict = {X: X_train, Y: Y_train, learning_rate: lr})
        if (i % 200 == 0):
            te_cost, te_acc = sess.run([cross_entropy, accuracy],
                           feed_dict = {X: X_test, Y: Y_test})
            print("epoch: %d, lr: %f, train cost: %f, train acc: %f, test cost: %f test acc %f" % (i, lr, tr_cost, tr_acc, te_cost, te_acc))
    
    acc = sess.run(accuracy, feed_dict = {X: X_test, Y: Y_test})
    print("Final accuracy: %f" % (acc))

---
# 4. Overfitting and Dropout
<img src="figs/13-comic5.png" style="width: 500px;"/>
You will have noticed that cross-entropy curves for test and training data start disconnecting after a couple thousand iterations. The learning algorithm works on training data only and optimises the training cross-entropy accordingly. It never sees test data so it is not surprising that after a while its work no longer has an effect on the test cross-entropy which stops dropping and sometimes even bounces back up. 
<img src="figs/14-overfit.png" style="width: 500px;"/>
This disconnect is usually labeled **overfitting** and when you see it, you can try to apply a regularisation technique called **dropout**. In dropout, at each training iteration, you drop random neurons from the network. You choose a probability `pkeep` for a neuron to be kept, usually between 50% and 75%, and then at each iteration of the training loop, you randomly remove neurons with all their weights and biases. Different neurons will be dropped at each iteration. When testing the performance of your network of course you put all the neurons back (`pkeep = 1`).
<img src="figs/15-dropout.png" style="width: 500px;"/>
TensorFlow offers a dropout function to be used on the outputs of a layer of neurons. It randomly zeroes-out some of the outputs and boosts the remaining ones by `1 / pkeep`. You can add dropout after each intermediate layer in the network now. 

In the following code, use the dropout between each layer during the training, and set the probability `pkeep` once to $50%$ and another time to $75%$ and compare their results.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

# neural network with 5 layers
#
# · · · · · · · · · ·          (input data, flattened pixels)       X [batch, 784]   
# \x/x\x/x\x/x\x/x\x/       -- fully connected layer (sigmoid)      W1 [784, 200]      B1[200]
#  · · · · · · · · ·                                                Y1_hat [batch, 200]
#   \x/x\x/x\x/x\x/         -- fully connected layer (sigmoid)      W2 [200, 100]      B2[100]
#    · · · · · · ·                                                  Y2_hat [batch, 100]
#     \x/x\x/x\x/           -- fully connected layer (sigmoid)      W3 [100, 60]       B3[60]
#      · · · · ·                                                    Y3_hat [batch, 60]
#       \x/x\x/             -- fully connected layer (sigmoid)      W4 [60, 30]        B4[30]
#        · · ·                                                      Y4_hat [batch, 30]
#         \x/               -- fully connected layer (softmax)      W5 [30, 10]        B5[10]
#          ·                                                        Y_hat [batch, 10]

# to reset the Tensorflow default graph
reset_graph()

def get_weight(name, shape):
    return tf.get_variable(name=name, 
                    shape=shape, 
                    dtype=tf.float32,
                    initializer=tf.initializers.random_uniform(-0.2, 0.2), 
                    trainable=True)

def get_bias(name, shape, val=0.1):
    return tf.get_variable(name=name, 
                    dtype=tf.float32, 
                    initializer=tf.constant(val, shape=shape),
                    trainable=True)
########################################
# define variables and placeholders
########################################
X = tf.placeholder(shape=[None, 28*28], dtype=tf.float32, name="input")
Y = tf.placeholder(shape=[None], dtype=tf.int32, name="labels")

# variable learning rate
learning_rate = tf.placeholder(shape=(), dtype=tf.float32, name="learning_rate")
keep_prob = tf.placeholder(shape=(), dtype=tf.float32, name="keep_prob")

# five layers and their number of neurons, i.e., 200, 100, 60, 30, and 10
# when using RELUs, make sure biases are initialised with small positive values, for example 0.1
W1 = get_weight("W1", [28*28, 200])
B1 = get_bias("B1", [200])

W2 = get_weight("W2", [200, 100])
B2 = get_bias("B2", [100])

W3 = get_weight("W3", [100, 60])
B3 = get_bias("B3", [60])

W4 = get_weight("W4", [60, 30])
B4 = get_bias("B4", [30])

W5 = get_weight("W5", [30, 10])
B5 = get_bias("B5", [10])

########################################
# build the model
########################################
XX = tf.reshape(X, [-1, 28**2])

Y1_hat = tf.nn.dropout(tf.nn.relu(tf.matmul(XX, W1) + B1), keep_prob=keep_prob)
Y2_hat = tf.nn.dropout(tf.nn.relu(tf.matmul(Y1_hat, W2) + B2), keep_prob=keep_prob)
Y3_hat = tf.nn.dropout(tf.nn.relu(tf.matmul(Y2_hat, W3) + B3), keep_prob=keep_prob)
Y4_hat = tf.nn.dropout(tf.nn.relu(tf.matmul(Y3_hat, W4) + B4), keep_prob=keep_prob)
Y_hat = tf.matmul(Y4_hat, W5) + B5

########################################
# defining the cost function
########################################
Y_one_hot = tf.cast(tf.one_hot(Y, 10), tf.float32)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y_one_hot, logits=Y_hat)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optimizer
########################################
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(cross_entropy)

correct = tf.nn.in_top_k(Y_hat, Y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float64))
########################################
# execute the model
########################################
init = tf.global_variables_initializer()

n_epochs = 5000
max_learning_rate = 0.005
min_learning_rate = 0.0001
decay_speed = 2000.0

def get_lr(min, max, decay_speed, i):
    return min + (max - min)*np.exp(-i / decay_speed)

with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        lr = get_lr(min_learning_rate, max_learning_rate, decay_speed, i)
        _, tr_cost, tr_acc = sess.run([train_step, cross_entropy, accuracy], 
                           feed_dict = {X: X_train, Y: Y_train, learning_rate: lr, keep_prob: 0.5})
        if (i % 200 == 0):
            te_cost, te_acc = sess.run([cross_entropy, accuracy],
                           feed_dict = {X: X_test, Y: Y_test, keep_prob: 1.0})
            print("epoch: %d, lr: %f, train cost: %f, train acc: %f, test cost: %f test acc %f" % (i, lr, tr_cost, tr_acc, te_cost, te_acc))
    
    acc = sess.run(accuracy, feed_dict = {X: X_test, Y: Y_test, keep_prob: 1.0})
    print("Final accuracy: %f" % (acc))

---
# 6. Convolutional Network
<img src="figs/16-comic6.png" style="width: 500px;"/>
In the previous sections, all pixels of images flattened into a single vector, which was a really bad idea. Handwritten digits are made of shapes and we discarded the shape information when we flattened the pixels. However, we can use **convolutional neural networks (CNN)** to take advantage of shape information. CNNs apply *a series of filters* to the raw pixel data of an image to extract and learn higher-level features, which the model can then use for classification. CNNs contains three components:
  - **Convolutional layers**: apply a specified number of convolution filters to the image. For each subregion, the layer performs a set of mathematical operations to produce a single value in the output feature map. Convolutional layers then typically apply a ReLU activation function to the output to introduce nonlinearities into the model.
  - **Pooling layers**: downsample the image data extracted by the convolutional layers to reduce the dimensionality of the feature map in order to decrease processing time. A commonly used pooling algorithm is max pooling, which extracts subregions of the feature map (e.g., 2x2-pixel tiles), keeps their maximum value, and discards all other values.
  - **Dense (fully connected) layers**: perform classification on the features extracted by the convolutional layers and downsampled by the pooling layers. In a dense layer, every node in the layer is connected to every node in the preceding layer.
  
Typically, a CNN is composed of a *stack of **convolutional modules*** that perform feature extraction. Each *module* consists of a *convolutional layer* followed by a *pooling layer*. The last convolutional module is followed by one or more dense layers that perform classification. The final dense layer in a CNN contains a single neuron for each target class in the model, with a softmax activation function to generate a value between 0-1 for each neuron. We can interpret the softmax values for a given image as relative measurements of how likely it is that the image falls into each target class.

Now, let us build a convolutional network for handwritten digit recognition. In this assignment, we will use the architecture shown in the following figure that has three convolutional layers, one fully-connected layer, and one softmax layer. Notice that the second and third convolutional layers have a stride of two that explains why they bring the number of output values down from 28x28 to 14x14 and then 7x7. A convolutional layer requires a weights tensor like `[4, 4, 3, 2]`, in which the first two numbers define the size of a filter (map), the third number shows the *depth* of the filter that is the number of *input channel*, and the last number shows the number of *output channel*. The output channel defines the number of times that we repeat the same thing with a different set of weights in one layer. In our implementation, we assume the output depth of first three convolutional layers, are 4, 8, 12, and the size of fully connected layer is 200.
<img src="figs/17-arch1.png" style="width: 600px;"/>

Convolutional layers can be implemented in TensorFlow using the `tf.nn.conv2d` function, which performs the scanning of the input image in both directions using the supplied weights. This is only the weighted sum part of the neuron. You still need to add a bias and feed the result through an activation function. The padding strategy that works here is to copy pixels from the sides of the image. All digits are on a uniform background so this just extends the background and should not add any unwanted shapes.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

# · · · · · · · · · ·      (input data, 1-deep)               X [batch, 28, 28, 1]
# @ @ @ @ @ @ @ @ @ @   -- conv. layer 5x5x1=>4 stride 1      W1 [5, 5, 1, 4]        B1 [4]
# ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                         Y1_hat [batch, 28, 28, 4]
#   @ @ @ @ @ @ @ @     -- conv. layer 5x5x4=>8 stride 2      W2 [5, 5, 4, 8]        B2 [8]
#   ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                           Y2_hat [batch, 14, 14, 8]
#     @ @ @ @ @ @       -- conv. layer 4x4x8=>12 stride 2     W3 [4, 4, 8, 12]       B3 [12]
#     ∶∶∶∶∶∶∶∶∶∶∶                                             Y3_hat [batch, 7, 7, 12] => reshaped to YY [batch, 7*7*12]
#      \x/x\x\x/        -- fully connected layer (relu)       W4 [7*7*12, 200]       B4 [200]
#       · · · ·                                               Y4_hat [batch, 200]
#       \x/x\x/         -- fully connected layer (softmax)    W5 [200, 10]           B5 [10]
#        · · ·                                                Y_hat [batch, 10]

# to reset the Tensorflow default graph
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(shape=[None, 28*28], dtype=tf.float32, name="input")
Y = tf.placeholder(shape=[None], dtype=tf.int32, name="labels")
XX = tf.reshape(X, [-1, 28, 28, 1])

learning_rate = tf.placeholder(shape=(), dtype=tf.float32, name="learning_rate")


# three convolutional layers with their channel counts, and a fully connected layer 
# (the last layer has 10 softmax neurons)
# the output depth of first three convolutional layers, are 4, 8, 12, and the size of fully connected
# layer is 200
W1 = get_weight(shape=[5,5,1,4], name="W1")
B1 = get_bias(shape=[4], name="B1")

W2 = get_weight(shape=[5,5,4,8], name="W2")
B2 = get_bias(shape=[8], name="B2")

W3 = get_weight(shape=[4,4,8,12], name="W3")
B3 = get_bias(shape=[12], name="B3")

W4 = get_weight(shape=[7*7*12, 200], name="W4")
B4 = get_bias(shape=[200], name="B4")

W5 = get_weight(shape=[200, 10], name="W5")
B5 = get_bias(shape=[10], name="B5")
########################################
# build the model
########################################
stride = 1  # output is 28x28
Y1_hat = tf.nn.relu(tf.nn.conv2d(XX, filter=W1, strides=[1, stride, stride, 1], padding="SAME") + B1) # use tf.nn.conv2d

stride = 2  # output is 14x14
Y2_hat = tf.nn.relu(tf.nn.conv2d(Y1_hat, filter=W2, strides=[1, stride, stride, 1], padding="SAME") + B2)

stride = 2  # output is 7x7
Y3_hat = tf.nn.relu(tf.nn.conv2d(Y2_hat, filter=W3, strides=[1, stride, stride, 1], padding="SAME") + B3)

# reshape the output from the third convolution for the fully connected layer
YY_hat = tf.reshape(Y3_hat, shape=[-1, 7*7*12])
Y4_hat = tf.nn.relu(tf.matmul(YY_hat, W4) + B4)
Y_hat = tf.matmul(Y4_hat, W5) + B5

########################################
# define the cost function
########################################
Y_one_hot = tf.cast(tf.one_hot(Y, 10), tf.float32)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y_one_hot, logits=Y_hat)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optmizer
########################################
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(cross_entropy)

correct = tf.nn.in_top_k(Y_hat, Y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float64))
########################################
# execute the model
########################################
init = tf.global_variables_initializer()

n_epochs = 2000
max_learning_rate = 0.005
min_learning_rate = 0.0001
decay_speed = 2000.0

def get_lr(min, max, decay_speed, i):
    return min + (max - min)*np.exp(-i / decay_speed)


print("Start training")
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        lr = get_lr(min_learning_rate, max_learning_rate, decay_speed, i)
        _, tr_cost, tr_acc = sess.run([train_step, cross_entropy, accuracy], 
                           feed_dict = {X: X_train, Y: Y_train, learning_rate: lr})
        if (i % 200 == 0):
            te_cost, te_acc = sess.run([cross_entropy, accuracy],
                           feed_dict = {X: X_test, Y: Y_test})
            print("epoch: %d, lr: %f, train cost: %f, train acc: %f, test cost: %f test acc %f" % (i, lr, tr_cost, tr_acc, te_cost, te_acc))
    
    acc = sess.run(accuracy, feed_dict = {X: X_test, Y: Y_test})
    print("Final accuracy: %f" % (acc))

# 7. Improve The Performance
A good approach to sizing your neural networks is to implement a network that is a little too constrained, then give it a bit more degrees of freedom and add dropout to make sure it is not overfitting. This ends up with a fairly optimal network for your problem. In the above model, we set the output channel to 4 in the first convolutional layer, which means that we repeat the same filter shape (but with different weights) four times. If we assume that those filters evolve during training into shape recognisers, you can intuitively see that this might not be enough for our problem. Handwritten digits are made from more than 4 elemental shapes. So let us bump up the filter sizes a little, and also increase the number of filters in our convolutional layers from 4, 8, 12 to 6, 12, 24 and then add dropout on the fully-connected layer. The following figure shows the new architecture you should build. Please complete the following code based on the given architecture and dropout technique.
<img src="figs/18-arch2.png" style="width: 600px;"/>

In [ ]:
# TODO: Replace <FILL IN> with appropriate code

# · · · · · · · · · ·    (input data, 1-deep)                 X [batch, 28, 28, 1]
# @ @ @ @ @ @ @ @ @ @ -- conv. layer 6x6x1=>6 stride 1        W1 [5, 5, 1, 6]        B1 [6]
# ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                         Y1_hat [batch, 28, 28, 6]
#   @ @ @ @ @ @ @ @   -- conv. layer 5x5x6=>12 stride 2       W2 [5, 5, 6, 12]        B2 [12]
#   ∶∶∶∶∶∶∶∶∶∶∶∶∶∶∶                                           Y2_hat [batch, 14, 14, 12]
#     @ @ @ @ @ @     -- conv. layer 4x4x12=>24 stride 2      W3 [4, 4, 12, 24]       B3 [24]
#     ∶∶∶∶∶∶∶∶∶∶∶                                             Y3_hat [batch, 7, 7, 24] => reshaped to YY [batch, 7*7*24]
#      \x/x\x\x/ ✞    -- fully connected layer (relu+dropout) W4 [7*7*24, 200]       B4 [200]
#       · · · ·                                               Y4_hat [batch, 200]
#       \x/x\x/       -- fully connected layer (softmax)      W5 [200, 10]           B5 [10]
#        · · ·                                                Y_hat [batch, 10]

# to reset the Tensorflow default graph
reset_graph()

########################################
# define variables and placeholders
########################################
X = tf.placeholder(shape=[None, 28*28], dtype=tf.float32, name="input")
Y = tf.placeholder(shape=[None], dtype=tf.int32, name="labels")
XX = tf.reshape(X, [-1, 28, 28, 1])

learning_rate = tf.placeholder(shape=(), dtype=tf.float32, name="learning_rate")
keep_prob = tf.placeholder(shape=(), dtype=tf.float32, name="keep_prob")


# three convolutional layers with their channel counts, and a fully connected layer 
# (the last layer has 10 softmax neurons)
# the output depth of first three convolutional layers, are 6, 12, 24, and the size of fully connected
# layer is 200
W1 = get_weight(shape=[5,5,1,6], name="W1")
B1 = get_bias(shape=[6], name="B1")

W2 = get_weight(shape=[5,5,6,12], name="W2")
B2 = get_bias(shape=[12], name="B2")

W3 = get_weight(shape=[4,4,12,24], name="W3")
B3 = get_bias(shape=[24], name="B3")

W4 = get_weight(shape=[7*7*24, 200], name="W4")
B4 = get_bias(shape=[200], name="B4")

W5 = get_weight(shape=[200, 10], name="W5")
B5 = get_bias(shape=[10], name="B5")


########################################
# build the model
########################################
stride = 1  # output is 28x28
Y1_hat = tf.nn.relu(tf.nn.conv2d(XX, filter=W1, strides=[1, stride, stride, 1], padding="SAME") + B1) # use tf.nn.conv2d

stride = 2  # output is 14x14
Y2_hat = tf.nn.relu(tf.nn.conv2d(Y1_hat, filter=W2, strides=[1, stride, stride, 1], padding="SAME") + B2)

stride = 2  # output is 7x7
Y3_hat = tf.nn.relu(tf.nn.conv2d(Y2_hat, filter=W3, strides=[1, stride, stride, 1], padding="SAME") + B3)

# reshape the output from the third convolution for the fully connected layer
YY_hat = tf.reshape(Y3_hat, shape=[-1, 7*7*24])
Y4_hat = tf.nn.dropout(tf.nn.relu(tf.matmul(YY_hat, W4) + B4), keep_prob)
Y_hat = tf.matmul(Y4_hat, W5) + B5
########################################
# define the cost function
########################################
Y_one_hot = tf.cast(tf.one_hot(Y, 10), tf.float32)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y_one_hot, logits=Y_hat)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optmizer
########################################
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(cross_entropy)

correct = tf.nn.in_top_k(Y_hat, Y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float64))
########################################
# execute the model
########################################
init = tf.global_variables_initializer()

n_epochs = 2000
max_learning_rate = 0.005
min_learning_rate = 0.0001
decay_speed = 2000.0

def get_lr(min, max, decay_speed, i):
    return min + (max - min)*np.exp(-i / decay_speed)


print("Start training")
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        print(i)
        lr = get_lr(min_learning_rate, max_learning_rate, decay_speed, i)
        _, tr_cost, tr_acc = sess.run([train_step, cross_entropy, accuracy], 
                           feed_dict = {X: X_train, Y: Y_train, learning_rate: lr, keep_prob: 0.75})
        if (i % 200 == 0):
            te_cost, te_acc = sess.run([cross_entropy, accuracy],
                           feed_dict = {X: X_test, Y: Y_test, keep_prob: 1.0})
            print("epoch: %d, lr: %f, train cost: %f, train acc: %f, test cost: %f test acc %f" % (i, lr, tr_cost, tr_acc, te_cost, te_acc))
    
    acc = sess.run(accuracy, feed_dict = {X: X_test, Y: Y_test, keep_prob: 1.0})
    print("Final accuracy: %f" % (acc))

---
# 8. Tensorflow Layers Module
The TensorFlow **layers** `tf.layers` module provides a high-level API that makes it easy to construct a neural network. It provides methods that facilitate: (i) the creation of dense (fully connected) layers and convolutional layers, (ii) adding activation functions, and (iii) applying dropout regularization. In this section use the module `tf.layers` to build the network you made in section 7.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
import tensorflow.layers as layers

# to reset the Tensorflow default graph
reset_graph()

# Define input
X = tf.placeholder(shape=[None, 28*28], dtype=tf.float32, name="input")
Y = tf.placeholder(shape=[None], dtype=tf.int32, name="labels")
XX = tf.reshape(X, [-1, 28, 28, 1])

learning_rate = tf.placeholder(shape=(), dtype=tf.float32, name="learning_rate")
keep_prob = tf.placeholder(shape=(), dtype=tf.float32, name="keep_prob")

# Define layers
c1 = layers.Conv2D(filters=6, 
        kernel_size=5, 
        strides=1,
        padding="SAME",
        kernel_initializer=tf.initializers.random_uniform(-0.2, 0.2),
        bias_initializer=tf.constant_initializer(0.1),
        activation = tf.nn.relu)

c2 = layers.Conv2D(filters=12, 
        kernel_size=5, 
        strides=2,
        padding="SAME",
        kernel_initializer=tf.initializers.random_uniform(-0.2, 0.2),
        bias_initializer=tf.constant_initializer(0.1),
        activation='relu')

c3 = layers.Conv2D(filters=24, 
        kernel_size=4, 
        strides=2, 
        padding="SAME", 
        kernel_initializer=tf.initializers.random_uniform(-0.2, 0.2),
        bias_initializer=tf.constant_initializer(0.1),
        activation='relu')

f1 = layers.Dense(units=200,
        activation='relu', 
        kernel_initializer=tf.initializers.random_uniform(-0.2, 0.2),
        bias_initializer=tf.constant_initializer(0.1))

dropout = layers.Dropout(1.0 - keep_prob)

f2 = layers.Dense(units=10,
        activation=None, 
        kernel_initializer=tf.initializers.random_uniform(-0.2, 0.2),
        bias_initializer=tf.constant_initializer(0.1))


Y1_hat = c1(XX)
Y2_hat = c2(Y1_hat)
Y3_hat = c3(Y2_hat)

Y4_hat = f1(tf.reshape(Y3_hat, shape=[-1, 7*7*24]))
YY_hat = dropout(Y4_hat)
Y_hat = f2(YY_hat)


Y_one_hot = tf.cast(tf.one_hot(Y, 10), tf.float32)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y_one_hot, logits=Y_hat)
cross_entropy = tf.reduce_mean(cross_entropy) * 100

########################################
# define the optmizer
########################################
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(cross_entropy)

correct = tf.nn.in_top_k(Y_hat, Y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float64))

init = tf.global_variables_initializer()
########################################
# Train
########################################
n_epochs = 2000
max_learning_rate = 0.005
min_learning_rate = 0.0001
decay_speed = 2000.0

def get_lr(min, max, decay_speed, i):
    return min + (max - min)*np.exp(-i / decay_speed)


print("Start training")
with tf.Session() as sess:
    sess.run(init)
    for i in range(n_epochs):
        print(i)
        lr = get_lr(min_learning_rate, max_learning_rate, decay_speed, i)
        _, tr_cost, tr_acc = sess.run([train_step, cross_entropy, accuracy], 
                           feed_dict = {X: X_train, Y: Y_train, learning_rate: lr, keep_prob: 0.75})
        if (i % 200 == 0):
            te_cost, te_acc = sess.run([cross_entropy, accuracy],
                           feed_dict = {X: X_test, Y: Y_test, keep_prob: 1.0})
            print("epoch: %d, lr: %f, train cost: %f, train acc: %f, test cost: %f test acc %f" % (i, lr, tr_cost, tr_acc, te_cost, te_acc))
    
    acc = sess.run(accuracy, feed_dict = {X: X_test, Y: Y_test, keep_prob: 1.0})
    print("Final accuracy: %f" % (acc))


---
# 9. Keras
Keras is a high-level API to build and train deep learning models. It's used for fast prototyping, advanced research, and production. `tf.keras` is TensorFlow's implementation of the Keras API specification. To work with Keras, you need to import `tf.keras` as part of your TensorFlow program setup.
```
import tensorflow as tf
from tensorflow.keras import layers
```
#### Build a model
In Keras, you assemble **layers** to build a model, i.e., a graph of layers. The most common type of model is a stack of layers: the `tf.keras.Sequential` model. For example, the following code builds a simple, fully-connected network (i.e., multi-layer perceptron):
```
model = tf.keras.Sequential()
# adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(64, activation='relu'))
# add another
model.add(layers.Dense(64, activation='relu'))
# add a softmax layer with 10 output units:
model.add(layers.Dense(10, activation='softmax'))
```
There are many `tf.keras.layers` available with some common constructor parameters:
* `activation`: set the activation function for the layer, which is specified by the name of a built-in function or as a callable object.
* `kernel_initializer` and `bias_initializer`: the initialization schemes that create the layer's weights (weight and bias).
* `kernel_regularizer` and `bias_regularizer`: the regularization schemes that apply the layer's weights (weight and bias), such as L1 or L2 regularization.

#### Train and evaluate
After you construct a model, you can configure its learning process by calling the `compile` method:
```
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
```
The method `tf.keras.Model.compile` takes three important arguments:
* `optimizer`: it specifies the training procedure, e.g., `tf.train.AdamOptimizer` and `tf.train.GradientDescentOptimizer`.
* `loss`: the cost function to minimize during optimization, e.g., mean square error (mse), categorical_crossentropy, and binary_crossentropy.
* `metrics`: used to monitor training, e.g., `accuracy`.

The next step after confiuring the model is to train it by calling the `model.fit` method and giving it training data as its input. After training the model you can call `tf.keras.Model.evaluate` and `tf.keras.Model.predict` methods to evaluate the inference-mode loss and metrics for the data provided or predict the output of the last layer in inference for the data provided, respectively.

You can read more about Keras [here](https://www.tensorflow.org/guide/keras).

In this task, please use Keras to rebuild the network you made in section 7.

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

# to reset the Tensorflow default graph
reset_graph()

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(6, kernel_size=(5, 5), strides=1, activation='relu', input_shape=input_shape))
model.add(Conv2D(12, (5, 5), strides=2, activation='relu'))
model.add(Conv2D(24, (4, 4),  strides=2, activation='relu'))
model.add(Flatten())
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

---
# 10. Implement LeNet-5
In this section, you should implement **LeNet-5** either using Tensorflow or Keras. Please take a look at its [paper](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf) before starting to implement it.
The LeNet-5 architecture is perhaps the most widely known CNN architecture. It was created by Yann LeCun in 1998 and widely used for handwritten digit recognition (MNIST). It is composed of the layers shown in the following table.
<img src="figs/19-letnet5.png" style="width: 600px;"/>
There are a few extra details to be noted:
* MNIST images are 28×28 pixels, but they are zero-padded to 32×32 pixels and normalized before being fed to the network. The rest of the network does not use any padding, which is why the size keeps shrinking as the image progresses through the network.
* The average pooling layers are slightly more complex than usual: each neuron computes the mean of its inputs, then multiplies the result by a learnable coefficient and adds a learnable bias term, then finally applies the activation function.
* Most neurons in layer C3 maps are connected to neurons in only three or four S2 maps (instead of all six S2 maps). See table 1 in the [paper](http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf) for details.
* The output layer is a bit special: instead of computing the dot product of the inputs and the weight vector, each neuron outputs the square of the Euclidian distance between its input vector and its weight vector. Each output measures how much the image belongs to a particular digit class. The cross-entropy cost function is now preferred, as it penalizes bad predictions much more, producing larger gradients and thus converging faster.

In [ ]:
# TODO: Build the LetNet-5 model, and test it on MNIST
import tensorflow as tf
import keras
from keras.datasets import mnist

def RBF(X, units, name="output"):
    input_size = X.get_shape()[-1]
    with tf.variable_scope(name):
        weights = tf.get_variable('kernel', shape = [units, input_size])
    X = tf.reshape(X, shape=[-1, 1, input_size])
    dif = X - weights
    dist = tf.pow(dif, 2)
    dist_sum = tf.reduce_sum(dist, axis=2)
    return tf.reshape(dist_sum, shape=(-1, units))


def sparse_conv2d(X, filters, kernel_size, strides, activation, filter_connections, seed=100):
    np.random.seed(seed)
    outputs = []
    [batch_size, channels, height, width] = X.get_shape()
    # Create a single conv-layer for a subset of input maps
    for i in range(filters):
        indices = np.random.randint(channels, size=filter_connections)
        # Slice out some input map
        slices = [X[:, indx, :, :] for indx in indices]
        filters = [tf.reshape(filt, shape=[-1, 1 , height, width]) for filt in slices]
        # Combine the selected ones and apply convolution with a single output
        X_ = tf.concat(filters, axis=1)
        Y_ = tf.layers.conv2d(X_, 1, kernel_size, strides, activation='tanh', data_format='channels_first')
        outputs.append(Y_)
    
    # Combine all the layers
    return tf.concat(outputs, axis=1)
    
def avg_pool_with_coff(X, pool_size, strides, name):
    [batch_size, channels, height, width] = X.get_shape()
    pooled = tf.layers.average_pooling2d(X, pool_size, strides,padding='valid', data_format='channels_first')
    
    with tf.variable_scope(name):
        weights = tf.get_variable('kernel', shape = [1, channels, 1, 1])
        bias = tf.get_variable('bias', shape = [1, channels, 1, 1])
        
    return tf.nn.tanh((pooled * weights) + bias)

# to reset the Tensorflow default graph
reset_graph()
batch_size = 128
num_classes = 10
epochs = 12
img_rows, img_cols = 28, 28

############# Set up the data ############
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols).astype(np.float64)
x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols).astype(np.float64)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

mean= np.mean(x_train)
std = np.std(x_train)

x_train -= mean
x_train /= std
x_test -= mean
x_test /= std

############ Build graph ######################
X = tf.placeholder(dtype=tf.float32, shape=[None, 1 , 28, 28])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])
learning_rate = tf.placeholder(dtype=tf.float64, shape=[])

c1 = tf.layers.conv2d(X, filters=6, kernel_size=5, strides=1, activation='tanh', data_format='channels_first', padding="SAME")
c1 = tf.Print(c1, [c1], message="C1 output is: ")
s2 = avg_pool_with_coff(c1, pool_size=2, strides=2, name='s1')

c3 = sparse_conv2d(s2, filters=16, kernel_size=5, strides=1, activation='tanh', filter_connections=3)
s4 = avg_pool_with_coff(c3, pool_size=2, strides=2, name='s4')

c5 = tf.layers.conv2d(s4, filters=120, kernel_size=5, strides=1, activation='tanh', data_format='channels_first')

f6 = tf.layers.dense(tf.layers.flatten(c5), units=84, activation='tanh')

Y_hat = RBF(f6, 10)
mse = tf.losses.mean_squared_error(Y, Y_hat)

cost = mse
############## Optimizer ###################
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_step = optimizer.minimize(cost)

correct = tf.nn.in_top_k(Y_hat, tf.argmax(Y, axis=1), 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float64))
############ Train ########################
n_epochs = 2000
max_learning_rate = 0.005
min_learning_rate = 0.0001
decay_speed = 2000.0
b_size = 128

def get_lr(min, max, decay_speed, i):
    return min + (max - min)*np.exp(-i / decay_speed)


print("Start training")
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    for i in range(n_epochs):
        lr = get_lr(min_learning_rate, max_learning_rate, decay_speed, i)
        for b_start in range(0, x_train.shape[0], b_size):
            print("At batch: ", b_start / b_size, end='\r')
            b_end = min(x_train.shape[0], b_start+b_size)
            _, tr_cost, tr_acc, y_hat = sess.run([train_step, cost, accuracy, Y_hat], 
                                           feed_dict = {
                                            X: x_train[b_start:b_end,:,:,:], 
                                            Y: y_train[b_start:b_end,:],
                                            learning_rate: lr}
                        )
        if (i % 1 == 0):
            te_cost, te_acc = sess.run([cost, accuracy],
                           feed_dict = {X: x_test, Y: y_test})
            print("epoch: %d, lr: %f, train cost: %f, train acc: %f, test cost: %f test acc %f" % (i, lr, tr_cost, tr_acc, te_cost, te_acc))
    
    acc = sess.run(accuracy, feed_dict = {X: X_test, Y: Y_test, keep_prob: 1.0})
    print("Final accuracy: %f" % (acc))

---
# 11. Implement AlexNet
In the last section, you should implement **AlexNet** either using Tensorflow or Keras. Again, please take a look at its [paper](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf) before start to implement it.
The AlexNet CNN architecture won the [ImageNet ILSVRC challenge](http://www.image-net.org/challenges/LSVRC/2012/) in 2012 by a large margin. It was developed by Alex Krizhevsky, Ilya Sutskever, and Geoffrey Hinton. It is quite similar to LeNet-5, only much larger and deeper, and it was the first to stack convolutional layers directly on top of each other, instead of stacking a pooling layer on top of each convolutional layer. The following table presents this architecture.
<img src="figs/20-alexnet.png" style="width: 600px;"/>
To train the model, we need a big dataset, however, in this assignment you are going to to assign the pretrained weights to your model, using `tf.Variable.assign`. You can download the pretrained weights from [bvlc_alexnet.npy](https://www.cs.toronto.edu/~guerzhoy/tf_alexnet/bvlc_alexnet.npy). This file is a NumPy array file created by the python. After you read this file, you will receive a python dictionary with a <key, value> pair for each layer. Each key is one of the layers names, e.g., `conv1`, and each value is a list of two values: (1) weights, and (2) biases of that layer. Part of the function to load the weights and biases to your model is given, and you need to complete it.

Here is what you see if you read and print the shape of each layer from the file:
```
weight_dic = np.load("bvlc_alexnet.npy", encoding="bytes").item()
for layer in weights_dic:
    print("-" * 20)
    print(layer)
    for wb in weights_dic[layer]:
        print(wb.shape)

#--------------------
# fc8
# (4096, 1000) # weights
# (1000,) # bias
#--------------------
# fc7
# (4096, 4096) # weights
# (4096,) # bias
#--------------------
# fc6
# (9216, 4096) # weights
# (4096,) # bias
#--------------------
# conv5
# (3, 3, 192, 256) # weights
# (256,) # bias
#--------------------
# conv4
# (3, 3, 192, 384) # weights
# (384,) # bias
#--------------------
# conv3
# (3, 3, 256, 384) # weights
# (384,) # bias
#--------------------
# conv2
# (5, 5, 48, 256) # weights
# (256,) # bias
#--------------------
# conv1
# (11, 11, 3, 96) # weights
# (96,) # bias
```


In [ ]:
# TODO: Replace <FILL IN> with appropriate code
import numpy as np
# to reset the Tensorflow default graph
reset_graph()

# build the AlexNet model
import tensorflow as tf
import tensorflow.layers as layers

# Used for splitting the convolutional layers
def conv2d_split(X, filters, kernel_size, strides, name, activation):
    input_channels = int(X.get_shape()[-1])
    with tf.variable_scope(name):
        weights = tf.get_variable('kernel', shape = [kernel_size, kernel_size, input_channels/2, filters])
        biases = tf.get_variable('bias', shape = [filters])
    
    conv = lambda x, w: tf.nn.conv2d(x, w, strides = [1, strides, strides, 1], padding="SAME")
    
    input_groups = tf.split(axis = 3, num_or_size_splits=2, value=X)
    weight_groups = tf.split(axis = 3, num_or_size_splits=2, value=weights)
    output_groups = [conv(x, w) for x,w in zip(input_groups, weight_groups)]

    output = tf.concat(axis = 3, values = output_groups) + biases
    
    return activation(output)
    

def AlexNet(X):
    c1 = layers.conv2d(X, filters=96, kernel_size=11, strides=4, padding="SAME", activation='relu', name='conv1')
    lrn1 = tf.nn.lrn(c1, depth_radius=2, alpha=1e-05, beta=0.75, bias=1.0)
    m1 = layers.max_pooling2d(lrn1, pool_size=3, strides=2, padding="VALID")
        
    c2 = conv2d_split(m1, filters=256, kernel_size=5, strides=1, name="conv2", activation=tf.nn.relu)
    lrn2 = tf.nn.lrn(c2, depth_radius=2, alpha=1e-05, beta=0.75, bias=1.0)
    m2 = layers.max_pooling2d(lrn2, pool_size=3, strides=2, padding="VALID")
    
    c3 = layers.conv2d(m2, filters=384, kernel_size=3, strides=1, padding="SAME", activation='relu', name='conv3')
    c4 = conv2d_split(c3, filters=384, kernel_size=3, strides=1, name="conv4", activation=tf.nn.relu)
    
    c5 = conv2d_split(c4, filters=256, kernel_size=3, strides=1, activation=tf.nn.relu, name='conv5')
    m5 = layers.max_pooling2d(c5, pool_size=3, strides=2, padding="VALID")
    
    f6 = layers.dense(layers.flatten(m5), 4096, activation='relu', name="fc6")
    f7 = layers.dense(f6, 4096, activation='relu', name="fc7")
    f8 = layers.dense(f7, 1000, activation='softmax', name="fc8")
    
    return f8

# load inital weights and biases to the model
def load_initial_weights(session):
    # load the weights into memory
    weights_dict = np.load('data/bvlc_alexnet.npy', encoding='bytes').item()
    var_dict = dict((v.name, v) for v in tf.global_variables())
    #print(v for v in var_dict.keys())
    # loop over all layer names stored in the weights dict
    for layer in weights_dict:
        with tf.variable_scope(layer, reuse=True):
            # loop over list of weights/biases and assign them to their corresponding tf variable
            for wb in weights_dict[layer]:
                # biases
                if len(wb.shape) == 1:
                    bias = var_dict[layer+'/bias:0']
                    session.run(bias.assign(wb))
                # weights
                else:
                    weight =  var_dict[layer+'/kernel:0']
                    session.run(weight.assign(wb))

#### Test the model
After building the AlexNet model, you can test it on different images and present the accuracy of the model. To do so, first you need to use **OpenCV** library to make the images ready to give as input to the model. OpenCV is a library used for image processing. Below you can see how to read an image file and pre-process it using OpenCV to give it to the model. However, you need to complete the code and test the accuracy of your model. The teset images (shown below) are available in the `test_images` folder.
<table width="100%">
<tr>
<td><img src="test_images/test_image1.jpg" style="width:200px;"></td>
<td><p align="center"><img src="test_images/test_image2.jpg" style="width:200px;"></td>
<td align="right"><img src="test_images/test_image3.jpg" style="width:200px;"></td>
</tr>

In [ ]:
# TODO: Replace <FILL IN> with appropriate code
# test the AlexNet model on the given images
import PIL
#import cv2
from scipy.misc import imread, imresize, imshow

#get list of all images
current_dir = os.getcwd()
image_path = os.path.join(current_dir, 'test_images')
img_files = [os.path.join(image_path, f) for f in os.listdir(image_path) if f.endswith('.jpg')]

#load all images
imgs = []
for f in img_files:
    imgs.append(imread(f))


reset_graph()

with tf.Session() as sess:
    X = tf.placeholder(shape=[None, 224, 224, 3], dtype=tf.float32, name='X')
    alex = AlexNet(X)
    load_initial_weights(sess)
    
    # loop over all images
    for i, image in enumerate(imgs):
        # convert image to float32 and resize to (227x227)
        img = imresize(image, (224, 224, 3)).astype(np.float32)
        
        import matplotlib.pyplot as plt
        plt.imshow(np.uint8(img))
        plt.show()
        # subtract the ImageNet mean
        # Mean subtraction per channel was used to center the data around zero mean for each channel (R, G, B).
        # This typically helps the network to learn faster since gradients act uniformly for each channel.
        imagenet_mean = np.array([104., 117., 124.], dtype=np.float32)
        img -= imagenet_mean
        
        # reshape as needed to feed into model
        img = img.reshape((1, 224, 224, 3))
        
        one_hot = sess.run(alex, feed_dict={X: img})
        print(np.argmax(one_hot))